# imports

In [2]:
import sys
import os
PROJECT_ROOT = os.path.abspath("..")  # ajusta según tu estructura
sys.path.append(PROJECT_ROOT)

import json
from pathlib import Path
from xml.etree.ElementInclude import include

import numpy as np
import pandas as pd
import joblib
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch import nn

from machine_learning.data_collectors import (
    build_ml_dataframe,
    build_supervised_dataset,
    time_split_masks,
    purged_ts_cv_splits,
    TARGET_HORIZONS,
    TARGET_LOOKBACKS,
)

from machine_learning.evaluators import eval_regression, eval_regression_extended

from python_scripts.LLM_analysis.preprocess_store_database import get_connection
from database_tier1 import TARGET_STOCKS

from typing import Tuple, Dict, List, Optional, Union, Iterable
from sklearn.preprocessing import StandardScaler
from machine_learning.artifacts import load_mlp_artifact, predict_artifact_to_compare
from machine_learning.evaluators import calculate_deadzone
from machine_learning.artifacts import discover_artifacts_by_horizon

import torch
import random

[DB_PATH_DEFAULT] /home/aprohack/Desktop/all_folders/Investings_project/app/data/stock_data.db
{'foreign_keys': 1, 'journal_mode': 'wal', 'synchronous': 1}


# load models by horizon

In [2]:
conn = get_connection()
symbols = TARGET_STOCKS

timeframe = "1Day"
# lookback = TARGET_LOOKBACKS[3]
horizon = TARGET_HORIZONS[2]

include_indicators = True

indicators_used = ['RSI_14', 'BBB_20_2.0', 'BBP_20_2.0', 'ATRr_14']

# indicators_used = ['SMA_20', 'EMA_50', 'RSI_14',
#        'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'BBL_20_2.0',
#        'BBM_20_2.0', 'BBU_20_2.0', 'BBB_20_2.0', 'BBP_20_2.0', 'ATRr_14']

include_economic_indicators = False
econ_indicator_names = ['CPI', 'UNEMPLOYMENT']

include_fmp = False
fmp_prefix = "fmp"
fmp_feature_names = []
keep_fmp_asof_date = False

base_feature_cols = ["open", "high", "low", "close", "volume", "trade_count", "vwap"] + indicators_used
horizon


20

In [3]:


artifacts = discover_artifacts_by_horizon(horizon)

for art in artifacts:
    print(art.get('run_dir'))

    

runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fmp0_trial4_convchan64-64-64-64-64-64-64
runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fmp0_trial7_convchan32-64-64-128-128-128
runs/mlp_1Day_lb120_h20_indicators0_econ0_fmp0_trial3_hs256-128_lr0.0001_do0.2_bs512
runs/mlp_1Day_lb120_h20_indicators0_econ0_fmp0_trial6_hs256-128_lr0.0001_do0.2_bs512
runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0_trial8_hs256-128_lr0.0001_do0.25_bs512
runs/mlp_1Day_lb252_h20_indicators0_econ0_fmp0_trial9_hs512-256-128_lr0.0001_do0.2_bs512
runs/mlp_1Day_lb252_h20_indicators1_econ0_fmp0_trial5_hs256-128_lr0.0001_do0.25_bs512
runs/mlp_1Day_lb60_h20_indicators1_econ0_fmp0_trial9_hs128-64_lr0.0001_do0.12_bs512


In [4]:



target_timestamps = [pd.to_datetime('2026-01-20 05:00:00+00:00')]

# chosen_run_dirs = [art.get('run_dir') for art in artifacts]
chosen_run_dirs = []
chosen_run_dirs = ['runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fmp0_trial4_convchan64-64-64-64-64-64-64',
                   'runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0_trial8_hs256-128_lr0.0001_do0.25_bs512',
                   'runs/mlp_1Day_lb252_h20_indicators1_econ0_fmp0_trial5_hs256-128_lr0.0001_do0.25_bs512',
                   'runs/mlp_1Day_lb60_h20_indicators1_econ0_fmp0_trial9_hs128-64_lr0.0001_do0.12_bs512'
                   ]
metrics_by_model = {}

for run_dir in chosen_run_dirs:
    df_gen, meta = predict_artifact_to_compare(
        run_dir,
        timeframe="1Day",
        symbols=TARGET_STOCKS,
        start="2023-01-01",
    )
    df_gen = df_gen[df_gen['target_timestamp'].isin(target_timestamps)]
    meta = meta[meta['target_timestamp'].isin(target_timestamps)]
    y_true = df_gen['y_true'].to_numpy()
    y_pred = df_gen['y_pred'].to_numpy()
    metrics = eval_regression_extended(y_true, y_pred, deadzone=calculate_deadzone(horizon), meta=meta)
    metrics_by_model[run_dir] = metrics


ValueError: Faltan lag cols para secuencia. Ej: ['open_lag0_lag251', 'high_lag0_lag251', 'low_lag0_lag251', 'close_lag0_lag251', 'volume_lag0_lag251', 'trade_count_lag0_lag251', 'open_lag1_lag251', 'high_lag1_lag251', 'low_lag1_lag251', 'close_lag1_lag251']

## generate metrics

In [ ]:
from machine_learning.artifacts import predict_ridge_by_run_dir, predict_torch_by_run_dir

def metrics_matrix(metrics_by_model: dict[str, dict]) -> pd.DataFrame:
    mat = pd.DataFrame(metrics_by_model)  # index=metric, columns=model
    # ordena: N arriba, luego lo demás
    if "N" in mat.index:
        ordered = ["N", 'DailyRankIC_mean', 'DailyRankIC_frac_pos', 'QuantileSpread_sharpe', 'DailyIC_mean', 'HitRate(sign,deadzone)', 'AUC(Sign)', 'QuantileSpread_std', 'DailyRankIC_std']
        order = ordered + [i for i in mat.index if i not in ordered]
        mat = mat.loc[order]
    return mat

test_mat = metrics_matrix(metrics_by_model)
test_mat


,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0_trial8_hs256-128_lr0.0001_do0.25_bs512
N,0.0
DailyRankIC_mean,NaN
DailyRankIC_frac_pos,NaN
QuantileSpread_sharpe,NaN
DailyIC_mean,NaN
"HitRate(sign,deadzone)",NaN
AUC(Sign),NaN
QuantileSpread_std,NaN
DailyRankIC_std,NaN
MAE,NaN


In [ ]:
from IPython.display import display
from matplotlib.colors import LinearSegmentedColormap

def style_metrics_by_row(
    mat: pd.DataFrame,
    exclude_rows=("N",),
    lower_is_better=(),   # e.g. ("QuantileSpread_std", "RMSE", ...)
    fmt="{:.4f}",
    row_limits: dict[str, tuple[float, float]] | None = None,  # <-- NUEVO
    clip: bool = True,                                       # <-- NUEVO
):
    mat_num = mat.copy().apply(pd.to_numeric, errors="coerce")

    # gmap: 0..1 por celda (0=peor/rojo, 1=mejor/verde)
    gmap = pd.DataFrame(index=mat_num.index, columns=mat_num.columns, dtype=float)

    row_limits = row_limits or {}

    for r in mat_num.index:
        row = mat_num.loc[r]

        # 1) Normalización con límites manuales si existen
        if r in row_limits:
            vmin, vmax = row_limits[r]
            denom = (vmax - vmin)
            if denom == 0:
                g = pd.Series(np.nan, index=row.index, dtype=float)
            else:
                g = (row - vmin) / denom
                if clip:
                    g = g.clip(0.0, 1.0)

        # 2) Si no hay límites, normaliza por min/max entre modelos (como antes)
        else:
            row_min = row.min()
            row_max = row.max()
            denom = (row_max - row_min)
            if pd.isna(denom) or denom == 0:
                g = pd.Series(np.nan, index=row.index, dtype=float)
            else:
                g = (row - row_min) / denom  # ya queda en 0..1

        # 3) Invertir escala si menor es mejor (después de normalizar)
        if r in lower_is_better:
            g = 1.0 - g

        gmap.loc[r] = g

    # (opcional) no colorear algunas filas
    rows_to_color = mat_num.index.difference(list(exclude_rows))

    # Colormap rojo → blanco → verde
    cmap = LinearSegmentedColormap.from_list(
        "red_white_green", ["#d73027", "#ffffff", "#1a9850"]
    )

    return (
        mat_num.style
            .format(fmt)
            .background_gradient(
                axis=None,
                cmap=cmap,
                gmap=gmap,
                subset=pd.IndexSlice[rows_to_color, :]  # type: ignore
            )
    )

row_limits = {
    "DailyRankIC_mean": (-0.02, 0.05),
    "DailyRankIC_frac_pos": (0.40, 0.60),
    "QuantileSpread_sharpe": (-2, 2.0),
    "DailyIC_mean": (-0.01, 0.02),
    "HitRate(sign,deadzone)": (0.4, 0.6),
    "AUC(Sign)": (0.4, 0.6),
}

styled = style_metrics_by_row(
    test_mat,
    exclude_rows=("N", "DailyIC_N", "N_deadzone", "DailyIC_tstat", "QuantileSpread_N",
                  "Conformal_avg_width(alpha=0.1)", "Conformal_avg_width(alpha=0.05)"),
    lower_is_better=("QuantileSpread_std", "DailyRankIC_std", "MAE", "MedianAE", "RMSE", "DailyIC_std"),
    row_limits=row_limits,
    clip=True,
)

display(styled)



,runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fmp0_trial4_convchan64-64-64-64-64-64-64,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0_trial8_hs256-128_lr0.0001_do0.25_bs512,runs/mlp_1Day_lb252_h20_indicators1_econ0_fmp0_trial5_hs256-128_lr0.0001_do0.25_bs512,runs/mlp_1Day_lb60_h20_indicators1_econ0_fmp0_trial9_hs128-64_lr0.0001_do0.12_bs512
N,82.0000,82.0000,82.0000,82.0000
DailyRankIC_mean,-0.0323,0.2314,-0.0916,0.1820
DailyRankIC_frac_pos,0.0000,1.0000,0.0000,1.0000
QuantileSpread_sharpe,nan,nan,nan,nan
DailyIC_mean,-0.0044,0.1692,-0.0753,0.1944
"HitRate(sign,deadzone)",0.6184,0.5789,0.5526,0.5526
AUC(Sign),0.5154,0.5923,0.4685,0.5931
QuantileSpread_std,nan,nan,nan,nan
DailyRankIC_std,nan,nan,nan,nan
MAE,0.0723,0.0740,0.0819,0.0761


## find models and compare metrics

In [ ]:
artifacts = discover_artifacts_by_horizon(20)

metrics_models = {}

for art in artifacts:
    metrics_models[art.get('run_dir')] = art.get('metrics').get('test')

test_mat = metrics_matrix(metrics_models)

styled = style_metrics_by_row(
    test_mat,
    exclude_rows=("N", "DailyIC_N", "N_deadzone", "DailyIC_tstat", "QuantileSpread_N",
                  "Conformal_avg_width(alpha=0.1)", "Conformal_avg_width(alpha=0.05)"),
    lower_is_better=("QuantileSpread_std", "DailyRankIC_std", "MAE", "MedianAE", "RMSE", "DailyIC_std"),
    row_limits=row_limits,
    clip=True,
)

display(styled)

,runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fmp0_trial4_convchan64-64-64-64-64-64-64,runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fmp0_trial7_convchan32-64-64-128-128-128,runs/mlp_1Day_lb120_h20_indicators0_econ0_fmp0_trial3_hs256-128_lr0.0001_do0.2_bs512,runs/mlp_1Day_lb120_h20_indicators0_econ0_fmp0_trial6_hs256-128_lr0.0001_do0.2_bs512,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0_trial8_hs256-128_lr0.0001_do0.25_bs512,runs/mlp_1Day_lb252_h20_indicators0_econ0_fmp0_trial9_hs512-256-128_lr0.0001_do0.2_bs512,runs/mlp_1Day_lb252_h20_indicators1_econ0_fmp0_trial5_hs256-128_lr0.0001_do0.25_bs512,runs/mlp_1Day_lb60_h20_indicators1_econ0_fmp0_trial9_hs128-64_lr0.0001_do0.12_bs512
N,27744.0000,27744.0000,8802.0000,8802.0000,18642.0000,8310.0000,18478.0000,18642.0000
DailyRankIC_mean,0.0437,0.0168,-0.0325,0.0030,0.0284,0.0245,0.0279,0.0281
DailyRankIC_frac_pos,0.6932,0.5959,0.4722,0.5185,0.5833,0.5882,0.6150,0.6053
QuantileSpread_sharpe,7.5846,-0.2636,-4.8649,4.3896,-2.7172,-7.3258,0.8504,4.0485
DailyIC_mean,0.0583,0.0091,0.0259,0.0540,0.0108,-0.0199,0.0279,0.0426
"HitRate(sign,deadzone)",0.5555,0.5488,0.5924,0.5809,0.5182,0.5777,0.5328,0.5248
AUC(Sign),0.5236,0.5167,0.4790,0.4883,0.5127,0.5142,0.5127,0.5097
QuantileSpread_std,0.0683,0.0363,0.0591,0.0811,0.0932,0.1459,0.0822,0.0642
DailyRankIC_std,0.1054,0.0946,0.1437,0.1153,0.1122,0.1438,0.1294,0.1138
MAE,0.0645,0.0646,0.0639,0.0637,0.0686,0.0649,0.0691,0.0678


# todays prediction

In [4]:
from machine_learning.artifacts import predict_artifact_live


pred_live = predict_artifact_live(
    run_dir="runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0_md6_lr0.03_mcw5_seed42",
    select="latest",
)

pred_live.sort_values("y_pred", ascending=False)


/home/aprohack/Desktop/all_folders/Investings_project/app/machine_learning/artifacts.py:3009: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{f}_lag{k}"] = g.shift(int(k))
/home/aprohack/Desktop/all_folders/Investings_project/app/machine_learning/artifacts.py:3009: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{f}_lag{k}"] = g.shift(int(k))
/home/aprohack/Desktop/all_folders/Investings_project/app/machine_learning/artifacts.py:3009: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calli

,symbol,timestamp,target_timestamp_est,y_pred,horizon,timeframe,run_dir,model_name,close,pred_price,rank,rank_pct,bucket
8,ASML,2026-01-26 05:00:00,2026-02-23 05:00:00,0.012916,20,1Day,runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0...,xgboost_regressor,1414.2900,1432.675783,1,0.012195,long
30,INTU,2026-01-26 05:00:00,2026-02-23 05:00:00,0.009988,20,1Day,runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0...,xgboost_regressor,564.9150,570.585478,2,0.024390,long
12,BTC,2026-01-26 05:00:00,2026-02-23 05:00:00,0.009685,20,1Day,runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0...,xgboost_regressor,38.6800,39.056421,3,0.036585,long
2,ADBE,2026-01-26 05:00:00,2026-02-23 05:00:00,0.009646,20,1Day,runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0...,xgboost_regressor,305.4650,308.425925,6,0.073171,long
68,WFC,2026-01-26 05:00:00,2026-02-23 05:00:00,0.009646,20,1Day,runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0...,xgboost_regressor,87.9450,88.797466,21,0.256098,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,XLP,2026-01-26 05:00:00,2026-02-23 05:00:00,0.008901,20,1Day,runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0...,xgboost_regressor,82.8000,83.540305,81,0.987805,short
39,LQD,2026-01-26 05:00:00,2026-02-23 05:00:00,0.008901,20,1Day,runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0...,xgboost_regressor,110.9399,111.931800,79,0.963415,short
35,KO,2026-01-26 05:00:00,2026-02-23 05:00:00,0.008901,20,1Day,runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0...,xgboost_regressor,72.8650,73.516477,78,0.951220,short
60,TLT,2026-01-26 05:00:00,2026-02-23 05:00:00,0.008901,20,1Day,runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0...,xgboost_regressor,88.2550,89.044077,80,0.975610,short


In [5]:
from machine_learning.artifacts import predict_artifact_live


pred_live = predict_artifact_live(
    run_dir="runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0_md4_lr0.07_mcw5_seed42",
    select="latest",
)

pred_live.sort_values("y_pred", ascending=False)


/home/aprohack/Desktop/all_folders/Investings_project/app/machine_learning/artifacts.py:3009: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{f}_lag{k}"] = g.shift(int(k))
/home/aprohack/Desktop/all_folders/Investings_project/app/machine_learning/artifacts.py:3009: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{f}_lag{k}"] = g.shift(int(k))
/home/aprohack/Desktop/all_folders/Investings_project/app/machine_learning/artifacts.py:3009: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calli

,symbol,timestamp,target_timestamp_est,y_pred,horizon,timeframe,run_dir,model_name,close,pred_price,rank,rank_pct,bucket
40,LRCX,2026-01-26 05:00:00,2026-04-20 05:00:00,0.037184,60,1Day,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,xgboost_regressor,223.6550,232.127893,2,0.024390,long
47,MU,2026-01-26 05:00:00,2026-04-20 05:00:00,0.037184,60,1Day,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,xgboost_regressor,391.7400,406.580586,3,0.036585,long
30,INTU,2026-01-26 05:00:00,2026-04-20 05:00:00,0.037184,60,1Day,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,xgboost_regressor,564.9150,586.316107,1,0.012195,long
3,AMAT,2026-01-26 05:00:00,2026-04-20 05:00:00,0.029703,60,1Day,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,xgboost_regressor,319.1625,328.784710,4,0.048780,long
4,AMD,2026-01-26 05:00:00,2026-04-20 05:00:00,0.029703,60,1Day,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,xgboost_regressor,253.3450,260.982923,5,0.060976,long
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,FCX,2026-01-26 05:00:00,2026-04-20 05:00:00,0.023973,60,1Day,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,xgboost_regressor,62.9200,64.446616,26,0.317073,neutral
21,DIS,2026-01-26 05:00:00,2026-04-20 05:00:00,0.023973,60,1Day,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,xgboost_regressor,111.6518,114.360786,25,0.304878,neutral
31,IWM,2026-01-26 05:00:00,2026-04-20 05:00:00,0.023973,60,1Day,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,xgboost_regressor,264.4200,270.835571,34,0.414634,neutral
34,KLAC,2026-01-26 05:00:00,2026-04-20 05:00:00,-0.034348,60,1Day,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,xgboost_regressor,1545.8500,1493.655124,81,0.987805,short


In [4]:

from machine_learning.artifacts import predict_artifact_live


pred_live = predict_artifact_live(
    run_dir="runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fmp0_trial4_convchan64-64-64-64-64-64-64",
    select="latest",
    start="2023-01-01"
)

pred_live.sort_values("y_pred", ascending=False)


/home/aprohack/Desktop/all_folders/Investings_project/app/machine_learning/artifacts.py:3009: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{f}_lag{k}"] = g.shift(int(k))
/home/aprohack/Desktop/all_folders/Investings_project/app/machine_learning/artifacts.py:3009: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{f}_lag{k}"] = g.shift(int(k))
/home/aprohack/Desktop/all_folders/Investings_project/app/machine_learning/artifacts.py:3009: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calli

ValueError: Faltan lag cols para secuencia. Ej: ['open_lag0_lag251', 'high_lag0_lag251', 'low_lag0_lag251', 'close_lag0_lag251', 'volume_lag0_lag251', 'trade_count_lag0_lag251', 'open_lag1_lag251', 'high_lag1_lag251', 'low_lag1_lag251', 'close_lag1_lag251']

In [ ]:
pred_live.sort_values("y_pred", ascending=False)

,symbol,timestamp,y_pred,target_timestamp_est,horizon,timeframe,run_dir,model_name,close,pred_price,rank,rank_pct,bucket
25,GOOGL,2026-01-21 05:00:00+00:00,0.024147,2026-02-18 05:00:00+00:00,20,1Day,runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fm...,cnn1d_regressor,328.0150,336.031852,1.0,0.012195,long
7,AMZN,2026-01-21 05:00:00+00:00,0.023602,2026-02-18 05:00:00+00:00,20,1Day,runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fm...,cnn1d_regressor,228.2600,233.711542,2.0,0.024390,long
0,AAPL,2026-01-21 05:00:00+00:00,0.023181,2026-02-18 05:00:00+00:00,20,1Day,runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fm...,cnn1d_regressor,246.2110,251.985093,3.0,0.036585,long
12,BTC,2026-01-21 05:00:00+00:00,0.021614,2026-02-18 05:00:00+00:00,20,1Day,runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fm...,cnn1d_regressor,39.1700,40.025854,4.0,0.048780,long
51,NVDA,2026-01-21 05:00:00+00:00,0.020448,2026-02-18 05:00:00+00:00,20,1Day,runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fm...,cnn1d_regressor,180.8866,184.623518,5.0,0.060976,long
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,BLK,2026-01-21 05:00:00+00:00,-0.009344,2026-02-18 05:00:00+00:00,20,1Day,runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fm...,cnn1d_regressor,1123.8900,1113.437837,78.0,0.951220,short
54,PG,2026-01-21 05:00:00+00:00,-0.009545,2026-02-18 05:00:00+00:00,20,1Day,runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fm...,cnn1d_regressor,146.1700,144.781393,79.0,0.963415,short
55,QCOM,2026-01-21 05:00:00+00:00,-0.009639,2026-02-18 05:00:00+00:00,20,1Day,runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fm...,cnn1d_regressor,156.2750,154.775900,80.0,0.975610,short
53,PEP,2026-01-21 05:00:00+00:00,-0.009745,2026-02-18 05:00:00+00:00,20,1Day,runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fm...,cnn1d_regressor,146.1750,144.757495,81.0,0.987805,short


# tcn

In [2]:
from machine_learning.artifacts import predict_artifact_live

# puedes pasar el root run_dir o el subdir final_model; se auto-resuelve a final_model
pred_live = predict_artifact_live(
    run_dir="runs/tcn_walkforward_2026-01-26_....",   # o ".../final_model"
    select="latest",
)

# ejemplo: ver top scores SOLO para horizon=20
pred_live[pred_live["horizon"] == 20].sort_values("y_pred", ascending=False)


FileNotFoundError: No encuentro artifact en runs/tcn_walkforward_2026-01-26_..... Esperaba model.pt/pipeline.joblib aquí o en subdir final_model/.

In [ ]:
df_gen, meta = predict_artifact_to_compare(
    run_dir="runs/tcn_walkforward_2026-01-26_....",  # o .../final_model
    timeframe="1Day",
    symbols=TARGET_STOCKS,
    start="2023-01-01",
)

for h in (5, 20, 60):
    df_h = df_gen[df_gen["horizon"] == h]
    meta_h = meta[meta["horizon"] == h]
    metrics = eval_regression_extended(
        df_h["y_true"].to_numpy(),
        df_h["y_pred"].to_numpy(),
        meta=meta_h,
        deadzone=calculate_deadzone(h),
    )
    print(h, metrics["DailyIC_mean"], metrics["RMSE"])


In [ ]:
target_timestamps = [pd.to_datetime('2026-01-20 05:00:00+00:00')]

h = 20
df_h = df_gen[(df_gen["horizon"] == h) & (df_gen["target_timestamp"].isin(target_timestamps))]
meta_h = meta[(meta["horizon"] == h) & (meta["target_timestamp"].isin(target_timestamps))]

metrics = eval_regression_extended(df_h["y_true"].to_numpy(), df_h["y_pred"].to_numpy(), meta=meta_h)


In [ ]:
from machine_learning.artifacts import predict_artifact_live

pred_live = predict_artifact_live(
    run_dir="runs/tcn_walkforward_2026-01-26_....",   # root o .../final_model
    select="latest",
)

# OJO: el TCN devuelve 3 horizontes => filtra:
pred_live[pred_live["horizon"] == 20].sort_values("y_pred", ascending=False)


In [ ]:
metrics_by_model = {}

for run_dir in chosen_run_dirs:
    df_gen, meta = predict_artifact_to_compare(
        run_dir,
        timeframe="1Day",
        symbols=TARGET_STOCKS,
        start="2023-01-01",
    )

    # compara horizon=20 para TODOS (los single horizon tendrán solo ese)
    h = 20
    df_h = df_gen[df_gen["horizon"] == h]
    meta_h = meta[meta["horizon"] == h]

    y_true = df_h["y_true"].to_numpy()
    y_pred = df_h["y_pred"].to_numpy()
    metrics = eval_regression_extended(y_true, y_pred, deadzone=calculate_deadzone(h), meta=meta_h)
    metrics_by_model[run_dir] = metrics


## ultimas 20 predicciones

In [6]:
pred_tail = predict_artifact_live(
    run_dir="runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0_trial8_hs256-128_lr0.0001_do0.25_bs512",
    select="tail",
    tail_n=20,
)
pred_tail


,symbol,timestamp,y_pred,target_timestamp_est,horizon,timeframe,run_dir,model_name,close,pred_price,rank,rank_pct,bucket
0,AAPL,2025-12-17 05:00:00+00:00,-0.010454,2026-01-14 05:00:00+00:00,20,1Day,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0...,mlp_regressor,271.84,269.012873,64.0,0.780488,neutral
1,ABBV,2025-12-17 05:00:00+00:00,-0.003030,2026-01-14 05:00:00+00:00,20,1Day,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0...,mlp_regressor,224.31,223.631263,52.0,0.634146,neutral
2,ADBE,2025-12-17 05:00:00+00:00,-0.015146,2026-01-14 05:00:00+00:00,20,1Day,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0...,mlp_regressor,354.66,349.328722,67.0,0.817073,neutral
3,AMAT,2025-12-17 05:00:00+00:00,0.004362,2026-01-14 05:00:00+00:00,20,1Day,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0...,mlp_regressor,248.27,249.355332,39.0,0.475610,neutral
4,AMD,2025-12-17 05:00:00+00:00,0.080695,2026-01-14 05:00:00+00:00,20,1Day,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0...,mlp_regressor,198.11,214.759105,2.0,0.024390,long
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635,XLRE,2026-01-15 05:00:00+00:00,-0.023410,2026-02-12 05:00:00+00:00,20,1Day,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0...,mlp_regressor,41.70,40.735129,71.0,0.865854,neutral
1636,XLU,2026-01-15 05:00:00+00:00,-0.024653,2026-02-12 05:00:00+00:00,20,1Day,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0...,mlp_regressor,43.61,42.548027,72.0,0.878049,neutral
1637,XLV,2026-01-15 05:00:00+00:00,-0.007339,2026-02-12 05:00:00+00:00,20,1Day,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0...,mlp_regressor,156.96,155.812349,50.0,0.609756,neutral
1638,XLY,2026-01-15 05:00:00+00:00,-0.018683,2026-02-12 05:00:00+00:00,20,1Day,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0...,mlp_regressor,122.70,120.428931,66.0,0.804878,neutral
